# LLM Lab 1
Shusrith S

PES1UG22AM155



# 1. Data source

Moby Dick; Or the Whale by Herman Melville





In [1]:
!wget https://www.gutenberg.org/cache/epub/2701/pg2701.txt

--2025-01-21 15:17:50--  https://www.gutenberg.org/cache/epub/2701/pg2701.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276288 (1.2M) [text/plain]
Saving to: ‘pg2701.txt’

pg2701.txt          100%[===================>]   1.22M  7.54MB/s    in 0.2s    

2025-01-21 15:17:50 (7.54 MB/s) - ‘pg2701.txt’ saved [1276288/1276288]



# 2. Preprocessing

The txt file is read and the text is converted to lowercase to avoid clashes between upper and lower case letters in the encoding. Regex is used to select only the alphabets and whitespaces for processing.



In [2]:
import re

with open("/content/pg2701.txt", "r") as f:
    text = f.read()
    text = text.lower()
    text = re.sub(r"[^a-z ]", "", text)

The corpus contains nearly 200k words which will be processed to build the encoding.

In [ ]:
words = text.split()
print(len(words))

196668


The text is converted to a list of characters for processing and only non whitespace characters are added to the corpus. We can see there are nearly 1 Million characters in the text initially.

In [ ]:
c = list(text)
corpus = []
for i in c:
    if i != " ":
        corpus.append(i)
len(corpus)

970386

A dictionary of words is created along with their counts for representation purposes. It shows the frequency of words which will be used for encoding.

In [ ]:
word_dict = {}
for i in corpus:
    if i in word_dict:
        word_dict[i] += 1
    else:
        word_dict[i] = 1
word_dict

{'t': 89874,
 'h': 63769,
 'e': 119351,
 'p': 17873,
 'r': 53593,
 'o': 70809,
 'j': 1178,
 'c': 23319,
 'g': 21287,
 'u': 27205,
 'n': 66782,
 'b': 17203,
 'k': 8228,
 'f': 21252,
 'm': 23626,
 'y': 17244,
 'd': 38840,
 'i': 66675,
 'w': 22557,
 'a': 79226,
 'l': 43351,
 's': 65138,
 'v': 8725,
 'z': 636,
 'x': 1064,
 'q': 1581}

# 3. Byte Pair Encoding

## First loop


The counter library is used extensively for counting the frequency of pairs of characters from the vocabulary.

We use a sliding window like technique to select neighbouring characters and create pairs of the form `(text[i], text[i + 1])`. Once these pairs are created, the counter library is called over the pairs, which aggregates all the pairs and gives us a count of the frequency of occurance of each pair. This is stored in a dictionary in the format `pair: frequency`.

In [ ]:
from collections import Counter
pairs = [(corpus[i], corpus[i + 1]) for i in range(len(corpus) - 1)]
pair_dict = Counter(pairs)
pair_dict

Counter({('t', 'h'): 33408,
         ('h', 'e'): 26991,
         ('e', 'p'): 2748,
         ('p', 'r'): 1941,
         ('r', 'o'): 5767,
         ('o', 'j'): 154,
         ('j', 'e'): 333,
         ('e', 'c'): 4345,
         ('c', 't'): 1416,
         ('t', 'g'): 395,
         ('g', 'u'): 714,
         ('u', 't'): 4395,
         ('t', 'e'): 8032,
         ('e', 'n'): 10039,
         ('n', 'b'): 879,
         ('b', 'e'): 4224,
         ('e', 'r'): 16298,
         ('r', 'g'): 692,
         ('g', 'e'): 2444,
         ('e', 'b'): 2584,
         ('b', 'o'): 2503,
         ('o', 'o'): 3077,
         ('o', 'k'): 996,
         ('k', 'o'): 304,
         ('o', 'f'): 7730,
         ('f', 'm'): 301,
         ('m', 'o'): 3228,
         ('o', 'b'): 1201,
         ('b', 'y'): 1420,
         ('y', 'd'): 567,
         ('d', 'i'): 4568,
         ('i', 'c'): 3015,
         ('c', 'k'): 2122,
         ('o', 'r'): 8438,
         ('r', 't'): 4471,
         ('e', 'w'): 4405,
         ('w', 'h'): 6002,
       

The ten most common pairs are printed for representation purposes. We can see `(t, h)` is the most frequent pair having 33400 occurences. This is now selected and merged and added to our vocabulary

In [ ]:
print(pair_dict.most_common(10))

[(('t', 'h'), 33408), (('h', 'e'), 26991), (('i', 'n'), 20143), (('e', 'r'), 16298), (('a', 'n'), 15485), (('e', 's'), 13486), (('h', 'a'), 12929), (('s', 't'), 12734), (('r', 'e'), 12246), (('n', 'd'), 11469)]


The corpus is now updated. We commbined all occurences of `t` followed `h` to give `th`in the corpus. We can see that the size of the corpus has reduced to around 930k.

In [ ]:
c = corpus
corpus = []
i = 0
while i < len(c):
    if c[i] == "t" and c [i + 1] == "h":
        corpus.append("th")
        i += 2
    else:
        corpus.append(c[i])
        i += 1
len(corpus)

936978

## Second loop

The same process is repeated, using the sliding window technique, creating pairs, finding the frequency of the pairs using Counter.

In [ ]:
from collections import Counter
pairs = [(corpus[i], corpus[i + 1]) for i in range(len(corpus) - 1)]
pair_dict = Counter(pairs)
pair_dict

Counter({('th', 'e'): 20753,
         ('e', 'p'): 2748,
         ('p', 'r'): 1941,
         ('r', 'o'): 5767,
         ('o', 'j'): 154,
         ('j', 'e'): 333,
         ('e', 'c'): 4345,
         ('c', 't'): 1381,
         ('t', 'g'): 395,
         ('g', 'u'): 714,
         ('u', 't'): 3915,
         ('t', 'e'): 8032,
         ('e', 'n'): 10039,
         ('n', 'b'): 879,
         ('b', 'e'): 4224,
         ('e', 'r'): 16298,
         ('r', 'g'): 692,
         ('g', 'e'): 2444,
         ('e', 'b'): 2584,
         ('b', 'o'): 2503,
         ('o', 'o'): 3077,
         ('o', 'k'): 996,
         ('k', 'o'): 304,
         ('o', 'f'): 7730,
         ('f', 'm'): 301,
         ('m', 'o'): 3228,
         ('o', 'b'): 1201,
         ('b', 'y'): 1420,
         ('y', 'd'): 567,
         ('d', 'i'): 4568,
         ('i', 'c'): 3015,
         ('c', 'k'): 2122,
         ('o', 'r'): 8438,
         ('r', 'th'): 2048,
         ('e', 'w'): 4405,
         ('w', 'h'): 6002,
         ('h', 'a'): 8402,
      

In the second loop we can see the most commonly occurence sequence is `th` followed by `e`, so this can be combined to give `the` and the vocabulary can be updated.

In [ ]:
print(pair_dict.most_common(1)[0])

(('th', 'e'), 20753)


## Consolidating into a function

The algorithm implemented so far can be consolidated into a function to perform Byte Pair Encoding.

All the steps done so far are utilized in the function to perform the entire encoding. The function accepts the preprocessed text as input and converts it to a list of characters. This list, excluding the whitespaces are appended into a list called `corpus`. This list contains all the characters of the corpus.

The vocabulary is a set of all characters in corpus. Initially it contains only single letter words but as the algorithm progresses and words are merged, the vocabulary increases. The for loop can be run as long as needed, for demonstration purposes, it is run 100 times in this example.


The loop starts with creating pairs using the sliding window method and using the Counter function to find the frequency of the pairs. Then the most frequent token is considered and merged to create a new token. We then iterate through the corpus and wherever the most frequent pair occur next to each other, we merge the occurences, creating a new merged corpus. If the new token is not in the existing vocabulary, it is appended to it.

Ideally, the algorithm should go on merging pairs as long as at least one token from the vocabulary exists in the corpus.
If none of the vocabulary tokens are found in the corpus, it implies that no further merges are possible or meaningful, and the loop terminates early with the break statement

`j` is used to keep count of the number of loops and stop the loop at 100 iterations for this demonstration.

In [17]:
from collections import Counter

def BPE(text):
    c = list(text)
    corpus = []
    for i in c:
        if i != " ":
            corpus.append(i)
    vocabulary = list(set(corpus))
    j = 0
    while j < 100:
        pairs = [(corpus[i], corpus[i + 1]) for i in range(len(corpus) - 1)]
        pair_dict = Counter(pairs)

        if not pair_dict:
            break

        most_common1, most_common2 = pair_dict.most_common(1)[0][0]
        new_token = most_common1 + most_common2
        merged_corpus = []
        i = 0
        while i < len(corpus):
            if i < len(corpus) - 1 and corpus[i] == most_common1 and corpus[i + 1] == most_common2:
                merged_corpus.append(new_token)
                i += 2
            else:
                merged_corpus.append(corpus[i])
                i += 1

        corpus = merged_corpus
        if new_token not in vocabulary:
            vocabulary.append(new_token)

        if not any(token in corpus for token in vocabulary):
            break
        j += 1
    return vocabulary

The function is run and we see that the given corpus contains 126 values when the algortihm is run for 100 iterations, meaning one new value was generated every iteration.

In [18]:
vocabulary_bpe = BPE(text)
len(vocabulary_bpe)

126

The vocabulary is printed out and we can see it has generated several meaning short words that usually occur in English, like `the`, `of`, `this`, `was`. There are also parts of words that occur together frequently like `id`, `un`,`em`, so by visual inspection we can say the vocabulary produced represents subwords that can be used for encoding.

In [19]:
vocabulary_bpe

['x',
 'i',
 's',
 'l',
 'q',
 'r',
 'j',
 'w',
 'y',
 'a',
 'f',
 'p',
 'u',
 'c',
 'b',
 'k',
 'd',
 'm',
 'n',
 'v',
 'e',
 't',
 'g',
 'o',
 'z',
 'h',
 'th',
 'the',
 'in',
 'an',
 'er',
 'es',
 'ed',
 'on',
 'at',
 'en',
 'is',
 'ou',
 'al',
 'or',
 'ing',
 'ar',
 'and',
 'as',
 'of',
 'to',
 'wh',
 'it',
 're',
 'st',
 'el',
 'om',
 'ch',
 'ow',
 'il',
 'se',
 'ad',
 'le',
 'ly',
 'im',
 'gh',
 'that',
 'be',
 'for',
 'bu',
 'ac',
 'he',
 'all',
 'id',
 'his',
 'ver',
 'ic',
 'ith',
 'ir',
 'ab',
 'ap',
 'am',
 'ent',
 'et',
 'ay',
 'em',
 'ol',
 'ofthe',
 'sh',
 'ot',
 'un',
 'with',
 'ec',
 'ter',
 'whal',
 'but',
 'ain',
 'ag',
 'ther',
 'av',
 'ur',
 'was',
 'this',
 'li',
 'ght',
 'od',
 'ess',
 'qu',
 'up',
 'ion',
 'sp',
 'fr',
 'est',
 'by',
 'inthe',
 'oun',
 'ro',
 'ke',
 'ul',
 'ri',
 'out',
 'ce',
 'ast',
 'ere',
 'ill',
 'you',
 'them',
 'ld',
 'one',
 'if',
 'lo']

## Algorithm to encode new text - Testing

This function takes a word and a list of Byte Pair Encoding  tokens, which are subword units, and attempts to encode the word by replacing adjacent character pairs in the word with the tokens. It iterates through the word's characters, checking for pairs that match any token in the `bpe_tokens` list. If a match is found, the pair is replaced with the token, and the second character of the pair is removed. This process continues for all tokens, progressively transforming the word into a list of merged subword units. The final result is a list that represents the encoded word using the BPE tokens.

In [ ]:
def encode_word(word, bpe_tokens):
    encoded_word = list(word)
    encoded_word = [i for i in encoded_word if i != ' ']
    for token in bpe_tokens:
        i = 0
        while i < len(encoded_word) - 1:
            if encoded_word[i] + encoded_word[i + 1] == token:
                encoded_word[i] = token
                del encoded_word[i + 1]
            else:
                i += 1
    return encoded_word

An unseen text `suzuki honda maruti` is encoded and we can see that it gets broken into subwords sufficiently

In [ ]:
tokenized = encode_word("suzuki honda maruti", encoding)
tokenized

['s', 'u', 'z', 'u', 'k', 'i', 'h', 'on', 'd', 'am', 'ar', 'u', 't', 'i']

# 4. Word piece encoding


We reuse the original text, remove the white spaces and make a list of all the characters, stored in `corpus`. There are 970k characters

In [5]:
c = list(text)
corpus = []
for i in c:
    if i != " ":
        corpus.append(i)
len(corpus)

970386

A dictionary called `freq_dict` is created. This dictionary contains characters and their frequencies and this dictionary will be used in the calculation of the score to select the pair to be merged.

In [6]:
freq_dict = {}
for i in corpus:
    if i in freq_dict:
        freq_dict[i] += 1
    else:
        freq_dict[i] = 1
freq_dict

{'t': 89874,
 'h': 63769,
 'e': 119351,
 'p': 17873,
 'r': 53593,
 'o': 70809,
 'j': 1178,
 'c': 23319,
 'g': 21287,
 'u': 27205,
 'n': 66782,
 'b': 17203,
 'k': 8228,
 'f': 21252,
 'm': 23626,
 'y': 17244,
 'd': 38840,
 'i': 66675,
 'w': 22557,
 'a': 79226,
 'l': 43351,
 's': 65138,
 'v': 8725,
 'z': 636,
 'x': 1064,
 'q': 1581}

Like BPE, we create pairs of characters using a sliding window like technique and find the frequency of occurence of these characters using Counter. The result is stored in `pair_dict`

In [ ]:
pairs = [(corpus[i], corpus[i + 1]) for i in range(len(corpus) - 1)]
pair_dict = Counter(pairs)
pair_dict

Counter({('t', 'h'): 33408,
         ('h', 'e'): 26991,
         ('e', 'p'): 2748,
         ('p', 'r'): 1941,
         ('r', 'o'): 5767,
         ('o', 'j'): 154,
         ('j', 'e'): 333,
         ('e', 'c'): 4345,
         ('c', 't'): 1416,
         ('t', 'g'): 395,
         ('g', 'u'): 714,
         ('u', 't'): 4395,
         ('t', 'e'): 8032,
         ('e', 'n'): 10039,
         ('n', 'b'): 879,
         ('b', 'e'): 4224,
         ('e', 'r'): 16298,
         ('r', 'g'): 692,
         ('g', 'e'): 2444,
         ('e', 'b'): 2584,
         ('b', 'o'): 2503,
         ('o', 'o'): 3077,
         ('o', 'k'): 996,
         ('k', 'o'): 304,
         ('o', 'f'): 7730,
         ('f', 'm'): 301,
         ('m', 'o'): 3228,
         ('o', 'b'): 1201,
         ('b', 'y'): 1420,
         ('y', 'd'): 567,
         ('d', 'i'): 4568,
         ('i', 'c'): 3015,
         ('c', 'k'): 2122,
         ('o', 'r'): 8438,
         ('r', 't'): 4471,
         ('e', 'w'): 4405,
         ('w', 'h'): 6002,
       

## Score calculation

The score is given by $$
\frac{\text{freq(character1, character2)}}{\text{freq(character1)} \times \text{freq(character2)}}
$$

For every element in the `pair_dict` dictionary, we find the frequency of element1 and element2 from the `freq_dict` dicitonary. `freq_dict` contains all the characters and their frequncies. Using this and the frequency of the pair, the wpe score is calculated and stored in `wpe_dict`. The pair with the highest score is then found using Python's builtin `max` function. We also store the frequency of occurence of this pair so that the `freq_dict` dictionary can be updated with the new pair. For the first loop, we can see `(z, z)` is the pair with the highest score with 43 occurences.


In [ ]:
def wpe_score(pair_dict, freq_dict):
    wpe_dict = {}

    for pair, count in pair_dict.items():
        elem1, elem2 = pair
        elem1_count = freq_dict[elem1]
        elem2_count = freq_dict[elem2]
        wpe_dict[pair] = [count / (elem1_count * elem2_count), count]
    max_pair, max_score = max(wpe_dict.items(), key=lambda x: x[1])
    return max_pair, max_score

wpe_score(pair_dict, freq_dict)

(('z', 'z'), [0.0001421487603305785, 43])

The new pair is added to `freq_dict` along with hits frequency

In [ ]:
freq_dict["zz"] = 43

The corpus is now updated with the new merged pair. All occurences of `z` followed by `z` is replaced with `zz` and the second `z` is deleted from the corpus.

In [ ]:
i = 0
while i < len(corpus):
    if corpus[i] == "z" and corpus[i + 1] == "z":
        corpus[i] = "zz"
        del corpus[i + 1]
        i += 2
    else:
        i += 1

# Second loop

We repeat this process again, with the updated corpus. We find all pairs in a sliding window and find their frequencies using Counter

In [ ]:
pairs = [(corpus[i], corpus[i + 1]) for i in range(len(corpus) - 1)]
pair_dict = Counter(pairs)
pair_dict

Counter({('t', 'h'): 33408,
         ('h', 'e'): 26991,
         ('e', 'p'): 2748,
         ('p', 'r'): 1941,
         ('r', 'o'): 5767,
         ('o', 'j'): 154,
         ('j', 'e'): 333,
         ('e', 'c'): 4345,
         ('c', 't'): 1416,
         ('t', 'g'): 395,
         ('g', 'u'): 714,
         ('u', 't'): 4395,
         ('t', 'e'): 8032,
         ('e', 'n'): 10039,
         ('n', 'b'): 879,
         ('b', 'e'): 4224,
         ('e', 'r'): 16298,
         ('r', 'g'): 692,
         ('g', 'e'): 2444,
         ('e', 'b'): 2584,
         ('b', 'o'): 2503,
         ('o', 'o'): 3077,
         ('o', 'k'): 996,
         ('k', 'o'): 304,
         ('o', 'f'): 7730,
         ('f', 'm'): 301,
         ('m', 'o'): 3228,
         ('o', 'b'): 1201,
         ('b', 'y'): 1420,
         ('y', 'd'): 567,
         ('d', 'i'): 4568,
         ('i', 'c'): 3015,
         ('c', 'k'): 2122,
         ('o', 'r'): 8438,
         ('r', 't'): 4471,
         ('e', 'w'): 4405,
         ('w', 'h'): 6002,
       

`wpe_score` function is called to find the next pair with the highest score and its frequency.

In [ ]:
wpe_score(pair_dict, freq_dict)

(('q', 'u'), [3.6734699096896025e-05, 1580])

The new pair is added to the `freq_dict` with its frequency

In [ ]:
freq_dict["qu"] = 1580

The corpus is updated with the new merged pair.

In [ ]:
i = 0
while i < len(corpus):
    if corpus[i] == "q" and corpus[i + 1] == "u":
        corpus[i] = "qu"
        del corpus[i + 1]
        i += 2
    else:
        i += 1

## Consolidated WPE

All the steps implemented so far are consolidated into a function. The `wpe_score` function uses the frequency of the pair and the frequency of its constituents to find the score for all pairs. The pair with the highest score is selected for merging.

The function `WPE` performs the algorithm for word piece encoding. Given a text corpus, it converts it to a list of characters. The whitespaces are removed to give us the starting corpus. The set of characters in this corpus gives us the vocabulary. The algorithm is run for 100 iterations in this example, but ideally it should be run till no new tokens can be added to the vocabulary.

The algorithm creates pairs of characters and finds the frequency of these pairs using Counter. Then the `wpe_score` function is used to find the pair with the highest score, which is then merged to create the new token. The corpus is updated with the new token, where existing occurrences of the pair is replaced with the new merged token. `freq_dict` is updated with the new token and its frequency. The frequency of the tokens with maximum frequency is then subtracted to update the removal of the pair tokens.

In [35]:
from collections import Counter, defaultdict

def wpe_score(pair_dict, freq_dict):
    wpe_dict = {}

    for pair, count in pair_dict.items():
        elem1, elem2 = pair
        elem1_count = freq_dict[elem1]
        elem2_count = freq_dict[elem2]
        wpe_dict[pair] = count / (elem1_count * elem2_count)

    if wpe_dict:
        max_pair = max(wpe_dict, key=wpe_dict.get)
        return max_pair, wpe_dict[max_pair], pair_dict[max_pair]
    else:
        return None, None, None


def WPE(text):
    corpus = [char for char in text if char != " "]
    freq_dict = Counter(corpus)
    vocabulary = list(freq_dict.keys())

    iterations = 0

    while iterations < 100:
        pairs = [(corpus[i], corpus[i + 1]) for i in range(len(corpus) - 1)]
        pair_dict = Counter(pairs)

        max_pair, max_score, max_count = wpe_score(pair_dict, freq_dict)
        if max_pair is None:
            break

        new_token = ''.join(max_pair)

        merged_corpus = []
        i = 0
        while i < len(corpus):
            if i < len(corpus) - 1 and (corpus[i], corpus[i + 1]) == max_pair:
                merged_corpus.append(new_token)
                i += 2
            else:
                merged_corpus.append(corpus[i])
                i += 1
        corpus = merged_corpus

        if new_token not in vocabulary:
            vocabulary.append(new_token)
        freq_dict[new_token] = max_count

        freq_dict[max_pair[0]] -= max_count
        freq_dict[max_pair[1]] -= max_count

        freq_dict[max_pair[0]] = max(freq_dict[max_pair[0]], 0)
        freq_dict[max_pair[1]] = max(freq_dict[max_pair[1]], 0)

        iterations += 1

    return vocabulary



The algorithm is run and we can see the new encoding vocabulary has 126 tokens, one each for each iteration.

In [36]:
vocabulary_wpe = WPE(text)
len(vocabulary_wpe)

126

In case of WPE, we can see that larger words are formed in the tokens and these words are usually repeating, with one new character added after each iteration. It can be noted that WPE needs more iterations in order to come up with meaningful tokens and 100 iterations used here are not sufficient. But the algorithm shows us that it can come up with a vocabulary better suited for a given corpus since the tokens formed are the ones with the highest frequency score in the corpus.

In [37]:
vocabulary_wpe

['t',
 'h',
 'e',
 'p',
 'r',
 'o',
 'j',
 'c',
 'g',
 'u',
 'n',
 'b',
 'k',
 'f',
 'm',
 'y',
 'd',
 'i',
 'w',
 'a',
 'l',
 's',
 'v',
 'z',
 'x',
 'q',
 'zz',
 'qu',
 'cq',
 'uzz',
 'puzz',
 'puzzl',
 'ypuzzl',
 'mypuzzl',
 'rypuzzl',
 'smypuzzl',
 'rypuzzli',
 'rypuzzlin',
 'rypuzzling',
 'ocq',
 'docq',
 'idocq',
 'vidocq',
 'fvidocq',
 'ofvidocq',
 'sofvidocq',
 'rsofvidocq',
 'irsofvidocq',
 'oirsofvidocq',
 'moirsofvidocq',
 'moirsofvidocqa',
 'moirsofvidocqan',
 'moirsofvidocqand',
 'moirsofvidocqandi',
 'moirsofvidocqandin',
 'moirsofvidocqandinf',
 'moirsofvidocqandinfo',
 'moirsofvidocqandinfor',
 'moirsofvidocqandinform',
 'rypuzzlinga',
 'rypuzzlingad',
 'huzz',
 'fuzz',
 'muzz',
 'dmuzz',
 'dmuzzl',
 'sfuzz',
 'ssfuzz',
 'ssfuzzi',
 'ssfuzzin',
 'ssfuzzing',
 'ssfuzzingu',
 'ssfuzzingup',
 'huzza',
 'huzzahuzza',
 'huzzap',
 'ghuzza',
 'huzzahuzzac',
 'huzzahuzzacr',
 'shuzzahuzzacr',
 'yshuzzahuzzacr',
 'yshuzzahuzzacri',
 'ighuzza',
 'dighuzza',
 'gdighuzza',
 'igdigh

## Encoding unseen words - Testing

This function takes a word and a list of Word Piece Encoding tokens. It iterates through the word's characters, checking for pairs that match any token in the `wpe_tokens` list. If a match is found, the pair is replaced with the token, and the second character of the pair is removed. This process continues for all tokens, progressively transforming the word into a list of merged subword units. The final result is a list that represents the encoded word using the WPE tokens.



In [41]:
def encode_word(word, wpe_tokens):
    encoded_word = list(word)
    encoded_word = [i for i in encoded_word if i != ' ']
    for token in wpe_tokens:
        i = 0
        while i < len(encoded_word) - 1:
            if encoded_word[i] + encoded_word[i + 1] == token:
                encoded_word[i] = token
                del encoded_word[i + 1]
            else:
                i += 1
    return encoded_word
encode_word("suzuki honda maruti", vocabulary_wpe)

['suz', 'u', 'k', 'i', 'hon', 'dam', 'ar', 'u', 't', 'i']

# 5. Comparison of Vocabularies

On comparing the two vocabularies, we can see that BPE produces tokens that are much smaller in length and represent the most frequently words occuring in natural language, like `the`, `is`, `of`. Whereas WPE tends to have larger tokens which are more likely to be seen in the given corpus. But since it is modelled on probabilties, it has a better tendency to work well on unseen and rare words in comparison to BPE.

In [28]:
vocabulary_bpe = sorted(vocabulary_bpe[26:])
vocabulary_wpe = sorted(vocabulary_wpe[26:])
set(vocabulary_wpe) - set(vocabulary_bpe)

{'adigdigdighuzzaohfor',
 'admuzzl',
 'boyshuzzahuzzacri',
 'broadmuzzl',
 'cq',
 'digdigdighuzza',
 'digdigdighuzzao',
 'digdigdighuzzaoh',
 'digdigdighuzzaohf',
 'digdigdighuzzaohfo',
 'digdigdighuzzaohfor',
 'digdighuzza',
 'dighuzza',
 'dmuzz',
 'dmuzzl',
 'docq',
 'dplumphuzzaporpois',
 'fuzz',
 'fvidocq',
 'gadigdigdighuzzaohfor',
 'gdigdighuzza',
 'gdighuzza',
 'ghuzza',
 'hisbroadmuzzl',
 'huzz',
 'huzza',
 'huzzahuzza',
 'huzzahuzzac',
 'huzzahuzzacr',
 'huzzap',
 'huzzapo',
 'huzzapor',
 'huzzaporp',
 'huzzaporpo',
 'huzzaporpoi',
 'huzzaporpois',
 'idocq',
 'igadigdigdighuzzaohfor',
 'igdigdighuzza',
 'igdighuzza',
 'ighuzza',
 'irsofvidocq',
 'isbroadmuzzl',
 'lumphuzzaporpois',
 'moirsofvidocq',
 'moirsofvidocqa',
 'moirsofvidocqan',
 'moirsofvidocqand',
 'moirsofvidocqandi',
 'moirsofvidocqandin',
 'moirsofvidocqandinf',
 'moirsofvidocqandinfo',
 'moirsofvidocqandinfor',
 'moirsofvidocqandinform',
 'mphuzzaporpois',
 'muzz',
 'myboyshuzzahuzzacri',
 'mypuzzl',
 'oadmuzzl'